In [59]:
from bs4 import BeautifulSoup
import requests
from dotenv import load_dotenv
import os
import urllib.request
import pprint
import json

headers = {
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
}
client_id = os.getenv("NAVER_CLIENT_ID") 
client_secret = os.getenv("NAVER_CLIENT_SECRET")

load_dotenv()


True

## 책 데이터 가져오기

In [62]:
base_url = "https://openapi.naver.com/v1/search/book.json"
def search_books(query):
    payload={
        "query":query,
        "display":50,
        "sort":"sim"
    }
    query_string = urllib.parse.urlencode(payload)
    url = f"{base_url}?{query_string}"

    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)

    response = urllib.request.urlopen(request)
    rescode = response.getcode()

    if(rescode==200):
        response_body=response.read().decode("utf-8")
        result=json.loads(response_body)
        return result["items"]



    

## 검색 데이터 저장

In [84]:
java_data=search_books("파이썬")

## 질문 1 : 검색어로 찾은 책 목록을 json 파일로 저장하기
-> data/book.json

In [85]:
def save_to_json(data,name):
    with open(f"data/{name}.json","w",encoding="utf-8") as file:
        json.dump(data,file)

In [86]:
save_to_json(java_data,"book")

## 질문 2 : books.json 파일을 Pandas DataFrame로 저장하기

In [108]:
import pandas as pd
def return_to_df(filename):
    return pd.read_json(filename)

df=return_to_df("data/book.json")

## 질문 3 : 검색어로 찾은 책 목록 출력하기

In [109]:
df.head()

,title,link,image,author,discount,publisher,pubdate,isbn,description
0,"혼자 만들면서 공부하는 파이썬 (37개 라이브러리와 API로 보고서, 차트, 게임,...",https://search.shopping.naver.com/book/catalog...,https://shopping-phinf.pstatic.net/main_528593...,문현일,23400,한빛미디어,20250214,9791169213271,“파이썬을 배워서 어디에 활용할 수 있을지 궁금했나요?”\n나의 신용카드 내역 분석...
1,혼자 공부하는 파이썬 (1:1 과외하듯 배우는 프로그래밍 자습서),https://search.shopping.naver.com/book/catalog...,https://shopping-phinf.pstatic.net/main_325076...,윤인성,19800,한빛미디어,20220601,9791162245651,혼자 해도 충분하다! 1:1 과외하듯 배우는 파이썬 프로그래밍 자습서\n\n『혼자 ...
2,Do it! 점프 투 파이썬 (중학생도 첫날부터 실습하는 초고속 입문서),https://search.shopping.naver.com/book/catalog...,https://shopping-phinf.pstatic.net/main_403540...,박응용,19800,이지스퍼블리싱,20230615,9791163034735,프로그래밍 분야 8년 연속 베스트셀러!\n《Do it! 점프 투 파이썬》 전면 개정...
3,밑바닥부터 시작하는 딥러닝 1(리마스터판) (파이썬으로 익히는 딥러닝 이론과 구현),https://search.shopping.naver.com/book/catalog...,https://shopping-phinf.pstatic.net/main_526155...,사이토 고키,23400,한빛미디어,20250124,9791169213387,딥러닝 분야 부동의 베스트셀러!\n머리로 이해하고 손으로 익히는 가장 쉬운 딥러닝 ...
4,파이썬,https://search.shopping.naver.com/book/catalog...,https://shopping-phinf.pstatic.net/main_324609...,홍의경,29440,생능출판,20220309,9788970506784,코딩 실력을 키우는 지름길은 실습이다.\n\n2008년 즈음에 검인정 중학교 교과서...


## 질문 4 : 검색어로 찾은 책 목록 중에서 가격이 2만원 이상인 책만 출력하기

In [110]:
cols_to_drop = ["image", "description", "isbn", "link"]

lower_20000 = (
    df[df["discount"] <= 20000]
    .drop(columns=cols_to_drop)
    .sort_values(by="discount", ascending=False)
    .reset_index(drop=True)
)

lower_20000.head()

,title,author,discount,publisher,pubdate
0,혼자 공부하는 파이썬 (1:1 과외하듯 배우는 프로그래밍 자습서),윤인성,19800,한빛미디어,20220601
1,Do it! 점프 투 파이썬 (중학생도 첫날부터 실습하는 초고속 입문서),박응용,19800,이지스퍼블리싱,20230615
2,클린 파이썬 (효과적인 파이썬 코딩 기법),수닐 카필,19800,에이콘출판,20210416
3,파이썬 정복 (파이썬 개발에 필요한 기본 지식을 모두 담은 파이썬 입문서),김상형,19800,한빛미디어,20180402
4,"파이썬 프로그래밍 (기초부터 실전까지 한 권에 끝내는, 개정판)",박영권,19800,시대인,20241010


## 질문 5 :  검색어로  찾은  책 목록 중에서 출판사가 인피니티북스인 책만 출력하기

In [111]:
from_infinite_books=df[df["publisher"]=="인피니티북스"].drop(columns=cols_to_drop)

from_infinite_books.head()

,title,author,discount,publisher,pubdate
7,파이썬,천인국,12000,인피니티북스,20170830
28,파이썬 플러스,최희식,23750,인피니티북스,20240731
49,디딤돌 파이썬 (구조적 프로그래밍으로 설명한 파이썬 기초),이찬수,26100,인피니티북스,20220823
